# Generation of Hardware

## Setup

In [ ]:
val ivy_path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(ivy_path)))

In [ ]:
import tetriski.pillars.core.{ArchitectureHierarchy, Connect, HardwareGenerator}
import tetriski.pillars.hardware.TopModule
import tetriski.pillars.archlib.BlockImmediate

## Construct the architecture of the top module

In this example, the top module consists of a single block (i.e., `BlockImmediate`).

In [ ]:
val arch = new ArchitectureHierarchy()

val blockImm = new BlockImmediate("blockImm")
arch.addBlock(blockImm)

arch.addInPorts(Array("in0", "in1"))
arch.addOutPorts(Array("out0"))

arch.addConnect(arch.term("in0") -> blockImm/"in0")
arch.addConnect(arch.term("in1") -> blockImm/"in1")
arch.addConnect(blockImm/"out0" -> arch.term("out0"))

arch.init()

## Construct the top module using `HardwareGenerator`

In [ ]:
val hardwareGenerator = new HardwareGenerator(arch, new Connect(arch.connectArray))

val topDesign = () => new TopModule(
    hardwareGenerator.pillarsModuleInfo, hardwareGenerator.connectMap,
    hardwareGenerator.regionList, blockImm.aluParams(0)
)

## Generate the Verilog Code

In [ ]:
println(chisel3.Driver.emitVerilog(topDesign()))